# Tracing LangChain Agents

This notebook shows you how to add **complete observability** to LangChain ReAct agents—tracing each step of the reasoning loop, capturing tool invocations with latency breakdowns, and understanding your agent's decision-making process.

**What You'll Learn:**
- Capture each iteration of thought → action → observation with Netra spans
- Monitor tool invocations with latency, inputs, outputs, and cost
- Understand why your agent made specific decisions using trace analysis
- Enrich traces with user IDs, session context, and custom attributes

**Note:** All company names (TaskBot, ShopFlow) and scenarios in this cookbook are entirely fictional and used for demonstration purposes only.

**Prerequisites:**
- Python >=3.10, <3.14
- OpenAI API key
- Netra API key ([Get started here](https://docs.getnetra.ai/quick-start/Overview))
- LangChain installed

## Step 0: Install Packages

In [ ]:
pip install netra-sdk langchain langchain-openai openai

## Step 1: Set Environment Variables

In [ ]:
import os
from getpass import getpass

os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API Key:")
os.environ["NETRA_API_KEY"] = getpass("Enter your Netra API Key:")
os.environ["NETRA_OTLP_ENDPOINT"] = getpass("Enter your Netra OTLP Endpoint:")

print("API keys configured!")


## Step 2: Initialize Netra for Agent Tracing

Netra provides auto-instrumentation for LangChain that captures agent execution automatically.

In [ ]:
from netra import Netra
from netra.instrumentation.instruments import InstrumentSet

# Initialize Netra with LangChain and OpenAI instrumentation
Netra.init(
    app_name="taskbot",
    headers=f"x-api-key={os.getenv('NETRA_API_KEY')}",
    environment="development",
    trace_content=True,
    instruments={InstrumentSet.OPENAI, InstrumentSet.LANGCHAIN},
)

print("Netra initialized with LangChain auto-instrumentation!")
print("Agent execution, LLM calls, and tool invocations will be traced automatically.")

## Step 3: Define Mock Data and Tools

Create a simple agent with multiple tools for a customer service scenario.

In [ ]:
from typing import Dict, List
from langchain.tools import tool
from langchain_openai import ChatOpenAI
from langchain.agents import create_agent

# Mock databases
TICKETS = {
    "TKT-001": {"id": "TKT-001", "subject": "Return policy question", "status": "open"},
    "TKT-002": {"id": "TKT-002", "subject": "Damaged item", "status": "open", "order_id": "ORD-12345"},
}

ORDERS = {
    "ORD-12345": {"id": "ORD-12345", "status": "delivered", "items": ["Headphones"], "total": 79.99},
}

KNOWLEDGE_BASE = [
    {"title": "Return Policy", "content": "Items can be returned within 30 days."},
    {"title": "Refund Processing", "content": "Refunds processed in 5-7 business days."},
]


@tool
def lookup_ticket(ticket_id: str) -> str:
    """Look up a ticket by its ID to get details about the issue."""
    ticket = TICKETS.get(ticket_id.upper())
    if not ticket:
        return f"No ticket found with ID: {ticket_id}"
    return f"Ticket {ticket['id']}: {ticket['subject']} (Status: {ticket['status']})"


@tool
def search_kb(query: str) -> str:
    """Search the knowledge base for information about policies or procedures."""
    query_lower = query.lower()
    results = [a for a in KNOWLEDGE_BASE if query_lower in a["title"].lower()]
    if not results:
        return "No relevant articles found."
    return "\n".join([f"**{a['title']}**: {a['content']}" for a in results])


@tool
def check_order_status(order_id: str) -> str:
    """Check the status of an order including shipping information."""
    order = ORDERS.get(order_id.upper())
    if not order:
        return f"No order found with ID: {order_id}"
    return f"Order {order['id']}: {order['status']}, Items: {order['items']}, Total: ${order['total']}"


@tool
def escalate_to_human(ticket_id: str, reason: str) -> str:
    """Escalate a ticket to a human operator for complex issues."""
    return f"Ticket {ticket_id} escalated. Reason: {reason}. A specialist will respond within 1 hour."


print("Tools defined!")

## Step 4: Create the ReAct Agent

Build the agent that uses the tools to respond to customer queries.

In [ ]:
# Initialize the LLM
model = ChatOpenAI(model="gpt-4o-mini", temperature=0, api_key=os.getenv("OPENAI_API_KEY"))

# Define tools
tools = [lookup_ticket, search_kb, check_order_status, escalate_to_human]

# Create the agent prompt
agent_prompt = """You are TaskBot, an AI assistant for ShopFlow e-commerce platform.

You help users with:
- Order status and tracking
- Return and refund requests
- Policy questions
- Escalating complex issues

Use tools to look up information before responding.
Escalate to human operators when the user is frustrated or you cannot resolve the issue."""

# Create the agent

agent = create_agent(
    model,
    tools,
    system_prompt=agent_prompt,
)



print("Agent created and ready!")

## Step 5: Define Agent Handler with Tracing

Wrap the agent execution with context tracking for user and session information.

In [ ]:
import uuid
from netra.decorators import agent

class TracedAgentSession:
    """Agent session with automatic tracing."""

    def __init__(self, user_id: str):
        self.user_id = user_id
        self.session_id = str(uuid.uuid4())
        self.turn_count = 0

    def handle_request(self, query: str) -> dict:
        """Handle a user request with full tracing."""
        self.turn_count += 1

        # Set context for tracing
        Netra.set_user_id(self.user_id)
        Netra.set_session_id(self.session_id)

        # Execute the agent
        print(f"\n=== Turn {self.turn_count} ===")
        print(f"User ({self.user_id}): {query}")

        try:
            result = agent.invoke({"input": query})
            response = result.get("output", "No response generated")
        except Exception as e:
            response = f"Error executing agent: {str(e)}"

        print(f"TaskBot: {response}")

        return {
            "user_id": self.user_id,
            "session_id": self.session_id,
            "turn": self.turn_count,
            "query": query,
            "response": response,
        }


print("Traced agent session class defined!")

## Step 6: Test with Sample Queries

Run the agent through different scenarios to see tracing in action.

In [ ]:
# Create a user session
session = TracedAgentSession(user_id="user-001")

# Test 1: Simple query - FAQ Lookup
print("\n" + "="*60)
print("TEST 1: Simple FAQ Lookup")
print("="*60)
result1 = session.handle_request("What is your return policy?")

In [ ]:
# Test 2: Order Status Query
print("\n" + "="*60)
print("TEST 2: Order Status Query")
print("="*60)
result2 = session.handle_request("Where is my order ORD-12345?")

In [ ]:
# Test 3: Multi-Step Query
print("\n" + "="*60)
print("TEST 3: Multi-Step Workflow")
print("="*60)
result3 = session.handle_request("I have ticket TKT-002 about a damaged item. Can you check the order status?")

In [ ]:
# Test 4: Escalation Scenario
print("\n" + "="*60)
print("TEST 4: Escalation Scenario")
print("="*60)
result4 = session.handle_request("I've been waiting forever and need urgent help! This is critical!")

## Step 7: Multi-Turn Conversation Tracking

Track a complete multi-turn conversation within a single session.

In [ ]:
# Create a new session for multi-turn conversation
conversation_session = TracedAgentSession(user_id="user-002")

print("\n" + "="*60)
print("MULTI-TURN CONVERSATION SESSION")
print("="*60)

# Turn 1: Start with a question
response1 = conversation_session.handle_request("What's your return policy?")

# Turn 2: Follow-up question
response2 = conversation_session.handle_request("How long do refunds take?")

# Turn 3: Specific order inquiry
response3 = conversation_session.handle_request("Can you check order ORD-12345?")

print("\n" + "="*60)
print(f"Session Complete: {conversation_session.turn_count} turns")
print(f"Session ID: {conversation_session.session_id}")
print("="*60)

## Step 8: Using Custom Span Attributes

Enrich traces with custom attributes for better filtering and analysis.

In [ ]:
from netra import SpanType

@tool
def lookup_ticket_with_attributes(ticket_id: str) -> str:
    """Look up a ticket with custom span attributes."""
    with Netra.start_span("ticket-lookup", as_type=SpanType.TOOL) as span:
        span.set_attribute("ticket_id", ticket_id)

        ticket = TICKETS.get(ticket_id.upper())

        if ticket:
            span.set_attribute("ticket_status", ticket["status"])
            span.set_attribute("found", True)
        else:
            span.set_attribute("found", False)

        if not ticket:
            return f"No ticket found with ID: {ticket_id}"

        return f"Ticket {ticket['id']}: {ticket['subject']} (Status: {ticket['status']})"


print("Custom span attributes pattern demonstrated!")
print("Use span.set_attribute(key, value) to add business context to traces.")

---

## What You'll See in the Dashboard

After running this notebook, check the Netra dashboard for:

- **Agent execution spans** showing the reasoning loop and tool selections
- **LLM calls** with prompts, completions, and token usage
- **Tool invocations** with inputs, outputs, and individual latencies
- **User and session IDs** attached to all spans for filtering conversation flows
- **Multi-turn conversation patterns** visible in trace hierarchy

## Key Debugging Patterns

| Problem | What to Look For in Traces |
|---------|----------------------------|
| Slow responses | High latency on specific tools or LLM calls |
| Wrong tool selection | Tool calls that don't match query intent |
| Infinite loops | Repeated identical tool calls |
| Missing information | Tools returning empty or error results |

## Documentation Links

- [Netra Documentation](https://docs.getnetra.ai)
- [Agent Tracing Guide](https://docs.getnetra.ai/Observability/Agents)
- [Decorators Guide](https://docs.getnetra.ai/Observability/Traces/decorators)
- [LangChain Integration](https://docs.getnetra.ai/Integrations/orchestrators/LangChain)

## See Also

- [Evaluate Agent Decisions](/Cookbooks/evaluation/evaluating-agent-decisions) - Add systematic evaluation to measure tool selection accuracy
- [Tracing RAG Pipeline](/Cookbooks/observability/tracing-rag-pipeline) - Apply similar patterns to RAG systems
- [Tracing CrewAI Pipelines](/Cookbooks/observability/tracing-crewai-pipelines) - Trace multi-agent workflows